<a href="https://colab.research.google.com/github/arahrooh31/UCLA_BE223C/blob/Keane_temp/get_dbt_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt

#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug 
!pip3 install -q SimpleITK
!pip3 install -q medpy

import SimpleITK as sitk  #for reading mha files
from medpy.io import load #mha files
import pydicom #to read dicom files




#
# Load data from google drive
#

drive.mount('/content/gdrive')




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
'''
GET DIR LISTING----------------------------------------------------------------
'''
def get_dirs(top_dir): 
    import os

    dir_list = os.scandir(top_dir)
    directories=[]
    file_list = {}
    for ii in dir_list:
        print('top dir is ',ii)
        file_list[ii.name]=[]
        directories.append(ii.name)
        #get file listings for each directory found
        full_name = os.path.join(top_dir,ii.name)
        print(ii,full_name)
        if (os.path.isdir(full_name) == 1):
            print('dir found')
            files = os.scandir(full_name) #os.path.join(top_dir,ii.name))
        else:
            files = full_name
            print('-----  ',ii)
        
            for jj in files:
                file_list[ii.name].append(jj.name)


    
    return directories, file_list

In [9]:
### Main level
top_dir = '/content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'


upper_dir =top_dir
main_dir, flist = get_dirs(upper_dir)

for ii in main_dir:
    print(ii)

<DirEntry 'license.html'> /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/license.html
-----   <DirEntry 'license.html'>


AttributeError: ignored

In [ ]:
'''
for key_val in flist.keys():
    print(flist[key_val])
    new_upper = os.path.join(upper_dir, key_val,str(flist[key_val][0]))
    temp_dir,temp_list= get_dirs(new_upper)
    print('---- ', temp_dir)

In [31]:
 ###
 ### Build links to dcm files
 ###
 print('upper dir = ',upper_dir)
 counter = 0
 patient_folder = {} #store dcm files
 for root, directories, filenames in os.walk(upper_dir): 
    #print('base is ', os.path.basename(directories))
    for directory in directories:
        #create dictionary entry for this directory. The files will be stored
        #within this
        patient_path = os.path.join(root, directory) #link to direct pt folder
        print('root, dir = ',root,patient_path)
        #for every directory found in the patient directory, get a listing of
        #files in each 
        patient_files = os.listdir(patient_path)
        for file in patient_files:
            if ('.dcm' in file):
                if (directory not in patient_folder.keys()):
                    patient_folder[directory] = []
                patient_folder[directory].append(patient_files)
    #for filename in filenames:  
    #    print('f= ',os.path.join(root,filename))
    #    counter = counter + 1
    #    if (counter > 10):
    #        break
    #if (counter > 10):
    #    break

upper dir =  /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT
root, dir =  /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00013
root, dir =  /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00064
root, dir =  /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00052
root, dir =  /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT /content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00023


In [32]:
for ii in patient_folder.keys():
    print(ii)

20566.000000-32081
20256.000000-46869
20257.000000-89476
20258.000000-36685
20259.000000-80607
2862.000000-19991
2863.000000-05408
2861.000000-84115
2864.000000-10096
19713.000000-61579
19710.000000-51654
19711.000000-14072
19712.000000-09387
2878.000000-74389
2877.000000-76997
2879.000000-50214
2880.000000-08806
10132.000000-67888
14783.000000-53289
14782.000000-91029
232.000000-42826
230.000000-30080
229.000000-39407
231.000000-99185
2665.000000-56730
2666.000000-26372
2668.000000-80894
2667.000000-29110
9672.000000-52664
9671.000000-81650
9673.000000-01012
9670.000000-95147
3592.000000-86534
3591.000000-25851
3593.000000-37029
3594.000000-09710
5578.000000-49286
5581.000000-30109
5580.000000-62386
5579.000000-02739
21991.000000-35241
21993.000000-40988
21992.000000-85380
21994.000000-28184
21886.000000-09498
21885.000000-00088
21888.000000-05060
21887.000000-83283
22027.000000-53436
22028.000000-60942
22029.000000-32173
22030.000000-92234
21224.000000-68917
21222.000000-76129
21223.

In [ ]:
### Open dcm file
###
from pydicom import dcmread
file = '/content/gdrive/My Drive/DBT_DATA/TEST_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00118/01-01-2000-DBT-S05386-MAMMO screening digital bilateral-70928/7697.000000-17471/1-1.dcm'


ds = dcmread(file)
print(ds.PatientName)
ds.file_meta.TransferSyntaxUID
plt.imshow(ds.pixel_array[0,:,:], cmap=plt.cm.bone) 


In [ ]:
np.shape(ds.pixel_array)
ds_slice = ds.pixel_array[0,:,:]
plt.imshow(ds_slice, cmap=plt.cm.jet)
plt.colorbar()